In [1]:
from playsound import playsound
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/startup.mp3')
from datetime import datetime as dt
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/JoeC/MetaTrader 5/terminal64.exe",login=1045610, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
def positioner(a,b,c,d,e):
    
    start = dt.now()
    
    riskusd = 5.00        
    symbol_info = mt5.symbol_info(a)
    point = mt5.symbol_info(a).point
    quote = a[3:]

    if quote == "USD":
        exchange = 1.00
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
    
    if quote == "JPY":
        stpips = abs(e - c)*100
        lot = round((((riskusd*exchange)/stpips)/10)/100,2)
    else:
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/stpips)/10,2)
        
    deviation = stpips/3
    
    if b == "LONG":
        price = mt5.symbol_info_tick(a).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
    elif b == "SHORT":
        price = mt5.symbol_info_tick(a).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
        
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)

    return request

In [4]:
def livepearls(h,i,j,k,l,m,threshold,threshold2,RATE,RATE2,spy,extraj):
    
    start = dt.now()
    
    if i == "LONG":
        lasttickb=mt5.symbol_info_tick(h).ask
        if lasttickb > extraj:
            RATE2 += 1
        else:
            RATE2 = 0
        if lasttickb > j:
            RATE += 1
        else:
            RATE = 0
    elif i == "SHORT":
        lasttickb=mt5.symbol_info_tick(h).bid
        if lasttickb < extraj:
            RATE2 += 1
        else:
            RATE2 = 0
        if lasttickb < j:
            RATE += 1
        else:
            RATE = 0
    
    if RATE >= threshold:
        request = positioner(h,i,j,k,l)
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("order_send failed, retcode={}".format(result.retcode))
            result_dict=result._asdict()
            for field in result_dict.keys():
                print("   {}={}".format(field,result_dict[field]))
                if field=="request":
                    traderequest_dict=result_dict[field]._asdict()
                    for tradereq_filed in traderequest_dict:
                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
        spy = "NO"
    elif RATE2 >= threshold2:
        request = positioner(h,i,j,k,l)
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("order_send failed, retcode={}".format(result.retcode))
            result_dict=result._asdict()
            for field in result_dict.keys():
                print("   {}={}".format(field,result_dict[field]))
                if field=="request":
                    traderequest_dict=result_dict[field]._asdict()
                    for tradereq_filed in traderequest_dict:
                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
        spy = "NO"
    
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)
    
    return RATE, RATE2, spy

In [5]:
def levelgrab(pair):
    
    levslist = []
    
    for x in pair:

        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H8, 0, 600)
        S = pd.DataFrame(Srates)
        S['time']=pd.to_datetime(S['time'], unit='s')
        S = S.set_index('time')
        S = S.assign(MOVE=(S['close'] - S['open']))
        SCM = np.average(abs(S['MOVE']))
        CNDL = (S.iloc[:, 3]) - (S.iloc[:, 0])
        S['CNDL'] = np.where(CNDL > 0, 1, -1)
        S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
        S = S[S.INFL != 0]
        S['INFL'] = S['CNDL']*-1
        S['LVL'] = np.where(S.INFL == 1, S.high, S.low)

        A = np.array(S.LVL)
        B = list(combinations(A, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = list(zip(Ba,Bb))
        table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
        table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
        table = table.sort_values(['a'], ascending=False)
        table = table.rename(columns={'b': 'Pings'})
        table = table.fillna(0)
        table = table[table['max'] != 0]
        pingsavg = np.average(table['Pings'])
        table = table[table['max'] > pingsavg]
        table = table.assign(level=table.index)
        levs = tuple(table['level'])
        
        levslist.append(levs)

    return levslist


In [6]:
## WATCHLIST = ['AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'AUD/USD', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'EUR/USD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'GBP/USD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'NZD/USD', 'USD/CAD', 'USD/CHF', 'USD/JPY', 'XAG/USD', 'XAU/USD', 'BCH/USD', 'BTC/USD', 'EOS/USD', 'ETH/USD', 'LTC/USD', 'XLM/USD', 'XRP/USD']

## FOREX = ## WATCHLIST = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

def brutus(pair,levslist):

    start = dt.now()
    purp = []
    l = 3
    lin = np.linspace(1,l,num=l)
    ia = 0
    for x in pair:
        
        levels = levslist[ia]
        
        print(levels)
        
        Drates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 3)
        D = pd.DataFrame(Drates)
        D['time']=pd.to_datetime(D['time'], unit='s')
        D = D.drop(columns=['tick_volume', 'spread', 'real_volume'])
        D = D.set_index('time')
        D = D.assign(MOVE=(D['close'] - D['open']))
        D = D.assign(STICK=(D['high'] - D['low']))
        D = D.assign(PSTICK=abs(D['MOVE']/D['STICK']))
        D['DIRECT'] = np.where(D.MOVE > 0, 'Bullish', 'Bearish')
        CM = np.average(abs(D['MOVE']))

        ANIMAL = []
        i = 0
        for t in lin: 
            if D.iloc[i,6] <= 1.1:
                if D.iloc[i,6] < 0.85:
                    if D.iloc[i,6] < 0.65:
                        if D.iloc[i,6] < 0.50:
                            if D.iloc[i,6] <= 0.30:
                                if D.iloc[i,6] <= 0.15:
                                    animal = 'Doji'
                                else:
                                    animal = 'Frog'
                            else:
                                animal = 'Cat'
                        else:
                            animal = 'Dog'
                    else:
                        animal = 'Horse'
                else:
                    animal = 'Shark'
            ANIMAL.append(animal)
            i += 1
        D['ANIMAL'] = ANIMAL

        GAUGE = []
        i = 0
        cndmid = (D.iloc[i,0]+D.iloc[i,3])/2
        stickmid = (D.iloc[i,1]+D.iloc[i,2])/2
        for t in lin: 
            if D.iloc[i,5] == 0:
                midpoint = D.iloc[i,0]
            else:
                midpoint = (cndmid - stickmid)/D.iloc[i,5]
            if midpoint >= 0.075:
                if midpoint >= 0.175:
                    if midpoint >= 0.25:
                        gauge = 'Top Saturated'
                    else:
                        gauge = 'Top Stacked'
                else:
                    gauge = 'Top Semi-Stacked'
            elif midpoint <= -0.075:
                if midpoint <= -0.175:
                    if midpoint <= -0.25:
                        gauge = 'Bottom Saturated'
                    else:
                        gauge = 'Bottom Stacked'
                else:
                    gauge = 'Bottom Semi-Stacked'
            else:
                gauge = 'Centered'
            GAUGE.append(gauge)
            i += 1
        D['GAUGE'] = GAUGE

        TREND = []
        DTrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 168)
        DT = pd.DataFrame(DTrates)
        DT['time']=pd.to_datetime(DT['time'], unit='s')
        DT = DT.drop(columns=['tick_volume', 'spread', 'real_volume'])
        DT = DT.set_index('time')
        DT = DT.assign(MOVE=(DT['close'] - DT['open']))
        movetrend = np.average(DT['MOVE'])
        TREND.append(movetrend)
        TREND.append(movetrend)
        TREND.append(movetrend)
        D['TREND'] = TREND

        READING=[]
        i = 0
        for t in lin: 
            if D.iloc[i,10] < 0:
                if D.iloc[i,9] == 'Top Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'HAMMER'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'MALLET'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Centered':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                else:
                    Read = D.iloc[i,8]
            elif D.iloc[i,10] > 0:
                if D.iloc[i,9] == 'Bottom Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'SHOOTING STAR'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'ASTEROID'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Centered':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                else:
                    Read = D.iloc[i,8]
            else:
                Read = D.iloc[i,8]
            i += 1
            READING.append(Read)
        D['READING'] = READING

        READING2=[0]
        linjj = np.linspace(1,2,num=2)
        i = 1
        for t in linjj: 
            ii = i - 1
            if D.iloc[i,10] < 0:
                if D.iloc[ii,7] == 'Bearish':
                    if D.iloc[i,3] > D.iloc[ii,1]:
                        if D.iloc[ii,11] == 'BRAKE':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'GLIDE':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'PLUMP DOJI':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'LONG-LEG DOJI':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,0] == D.iloc[ii,3]:
                        if D.iloc[i,2] == D.iloc[ii,2]:
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,3] > D.iloc[ii,0]:
                                    Read2 = 'TWEEZERS BOTTOM'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,3] > (D.iloc[ii,0]+D.iloc[ii,3])/2:
                        if D.iloc[i,0] < D.iloc[ii,3]:
                            if D.iloc[ii,7] == 'Bullish':
                                Read2 = 'PIERCING PATTERN'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            elif D.iloc[i,10] > 0:
                if D.iloc[ii,7] == 'Bullish':
                    if D.iloc[i,3] < D.iloc[ii,2]:
                        if D.iloc[ii,11] == 'BRAKE':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'GLIDE':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'PLUMP DOJI':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'LONG-LEG DOJI':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,3] == D.iloc[ii,0]:
                    if D.iloc[i,1] == D.iloc[ii,1]:
                        if D.iloc[i,7] == 'Bearish':
                            if D.iloc[i,3] < D.iloc[ii,0]:
                                Read2 = 'TWEEZERS TOP'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,3] < (D.iloc[ii,0]+D.iloc[ii,3])/2:
                    if D.iloc[i,0] > D.iloc[ii,3]:
                        if D.iloc[ii,7] == 'Bearish':
                            Read2 = 'DARK CLOUD COVER'
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            else:
                Read2 = 0
            i += 1
            READING2.append(Read2)
        D['READING2'] = READING2

        P = D.iloc[-2,:]

        OTESTLVL = 0
        for u in levels:
            if abs(P[0]-u) <= CM:
                OTESTLVL = u
        CTESTLVL = 0
        for u in levels:
            if abs(P[3]-u) <= CM:
                CTESTLVL = u
        BREAKLVL = 0
        for u in levels:
            if P[7] == 'Bullish':
                if P[3] > (u+CM):
                    if P[0] < u:
                        BREAKLVL = u
                    elif OTESTLVL == u:
                        BREAKLVL = u
            elif P[7] == 'Bearish':
                if P[3] < (u-CM):
                    if P[0] > u:
                        BREAKLVL = u
                    elif OTESTLVL == u:
                        BREAKLVL = u
      
        F = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M15, 0, 200)
        x1 = np.linspace(1, 200, num=200)
        y1 = F['close']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
        wholeslope = slope

        if wholeslope > 0:
            if P[10] > 0:
                ditrend = 1
            elif P[10] < 0:
                ditrend = -1
            else:
                ditrend = 0
        elif wholeslope < 0:
            if P[10] < 0:
                ditrend = 1
            elif P[10] > 0:
                ditrend = -1
            else:
                ditrend = 0
        else:
            ditrend = 0

        if P[7] == 'Bullish':
            if P[10] > 0:
                dstrend = 1
            elif P[10] < 0:
                dstrend = -1
            else:
                dstrend = 0
        elif P[7] == 'Bearish':
            if P[10] < 0:
                dstrend = 1
            elif P[10] > 0:
                dstrend = -1
            else:
                dstrend = 0
        else:
            dstrend = 0

        ypa = ['JACK', 'GLIDE']
        grpa = ['MARIBOZU', 'TUBE', 'BOOST']
        goldpa = ['HAMMER', 'SHOOTING STAR', 'DRAGONFLY DOJI', 'GRAVESTONE DOJI', 'LONG-LEG DOJI']
        silvpa = ['PLUMP DOJI', 'MALLET', 'ASTEROID', 'DIPPER', 'SKILLET']
        goldpa2 = ['BULLISH ENGULFING', 'BEARISH ENGULFING', 'TWEEZERS TOP', 'TWEEZERS BOTTOM', 'DARK CLOUD COVER', 'PIERCING PATTERN']

        color = 'white'
        if BREAKLVL != 0:
            if CTESTLVL == 0:
                if dstrend == 1:
                    color = 'pearl'
                elif dstrend == -1:
                    color = 'black'
        elif BREAKLVL == 0:
            if CTESTLVL != 0:
                if OTESTLVL == CTESTLVL:
                    if P[12] == 0:
                        color = 'gray'
                elif OTESTLVL == 0:
                    if P[12] == 0:
                        if dstrend == 1:
                            color = 'blue'
                        elif dstrend == -1:
                            color = 'purple'
                    elif P[12] != 0:
                        for ugo2 in goldpa2:
                            if P[12] == ugo2:
                                color = 'gold'
                            elif P[12] == 0:
                                if P[11] != 0:
                                    for ugo in goldpa:
                                        if P[11] == ugo:
                                            color = 'gold'
                elif OTESTLVL == 0:
                    if P[12] == 0:
                        for uy in ypa:
                            if P[11] == uy:
                                if dstrend == 1:
                                    color = 'yellow'
                                elif dstrend == -1:
                                    color = 'orange'
                        for ug in grpa:
                            if P[11] == ug:
                                if dstrend == 1:
                                    color = 'green'
                                elif dstrend == -1:
                                    color = 'red'
        
        if color == 'purple':
            coltrade = 'TRADE'
        elif color == 'gold':
            coltrade = 'TRADE'
        else:
            coltrade = 0
                
        if coltrade == 'TRADE':
            
            Hrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H2, 0, 25)
            H = pd.DataFrame(Hrates)
            hlin = np.linspace(1,25,num=25)
            SLOPE = []
            i = 0
            for t in hlin:
                ii = i - 1
                if i == 0:
                    SLOPE.append(0)
                else:
                    slope = H.iloc[i,4] - H.iloc[ii,4]
                    SLOPE.append(slope)
                i += 1
            SLOPE2 = []
            i = 0
            for t in hlin:
                ii = i - 1
                if i == 0:
                    SLOPE2.append(0)
                else:
                    slope2 = SLOPE[i] - SLOPE[ii]
                    SLOPE2.append(slope2)
                i += 1
            CONCAVE = []
            i = 0
            for t in hlin:
                if SLOPE2[i] == 0:
                    CONCAVE.append('INFL')
                elif SLOPE2[i] > 0:
                    CONCAVE.append('UP')
                else:
                    CONCAVE.append('DOWN')
                i += 1

            AXCONCAVE = CONCAVE[-1]
            if P[10] > 0:
                if AXCONCAVE == 'UP':
                    contrade = 'TRADE'
                else:
                    contrade = 0
            elif P[10] < 0:
                if AXCONCAVE == 'DOWN':
                    contrade = 'TRADE'
                else:
                    contrade = 0
            else:
                contrade = 0

            if contrade == 'TRADE':
                
                if P[10] > 0:
                    position = 'LONG'
                    entry = CTESTLVL + CM
                elif P[10] < 0:
                    position = 'SHORT'
                    entry = CTESTLVL - CM
                else:
                    position = 0
                    entry = 0

                cend = int(len(levels)-1)
                if position == 'LONG':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex - 1)
                    slindex = int(clindex + 1)
                    if clindex == 0:
                        stoploss = levels[slindex]
                        sldist = entry - stoploss
                        takeprofit = entry + sldist
                    elif clindex == cend:
                        takeprofit = levels[tpindex]
                        tpdist = takeprofit - entry
                        stoploss = entry - tpdist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                elif position == 'SHORT':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex + 1)
                    slindex = int(clindex - 1)
                    if clindex == 0:
                        takeprofit = levels[tpindex]
                        tpdist = entry - takeprofit
                        stoploss = entry + tpdist
                    elif clindex == cend:
                        stoploss = levels[slindex]
                        sldist = stoploss - entry
                        takeprofit = entry - sldist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                else:
                    takeprofit = 0
                    stoploss = 0
                
                shrink = 0.90
                if position == 'LONG':
                    tpdist = takeprofit - entry
                    takeprofit = entry + tpdist*shrink
                elif position == 'SHORT':
                    tpdist = entry - takeprofit
                    takeprofit = entry - tpdist*shrink

                if position == 'LONG':
                    tpdist = takeprofit - entry
                    sldist = entry - stoploss
                    if sldist > tpdist:
                        stoploss = entry - tpdist
                elif position == 'SHORT':
                    tpdist = entry - takeprofit
                    sldist = stoploss - entry
                    if sldist > tpdist:
                        stoploss = entry + tpdist
                
                prp = (x,position,entry,takeprofit,stoploss,CM)
                purp.append(prp)
        ia += 1

    purps = pd.DataFrame(purp, columns =['x','position','entry','takeprofit','stoploss','CM']) 
    purps['RATE'] = 0
    purps['RATE2'] = 0
    purps['spy'] = "YES"

    purpcnt = int(purps['x'].count())
    purplin = np.linspace(1,purpcnt,num=purpcnt)
    
    EJ = []
    i = 0
    for t in purplin:
        if purps.iloc[i,1] == "LONG":
            extrajj = purps.iloc[i,2] + purps.iloc[i,5]
        elif purps.iloc[i,1] == "SHORT":
            extrajj = purps.iloc[i,2] - purps.iloc[i,5]
        EJ.append(extrajj)
        i += 1
    purps['extraj'] = EJ
    
    print(purps)
    
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)
    
    watchers = purps['spy']
    ite = 0
    while any(s == "YES" for s in watchers):
        i = 0
        for t in purplin:
            if purps.iloc[i,8] == "YES":
                Xpair = purps.iloc[i,0]
                position = purps.iloc[i,1]
                entry = purps.iloc[i,2]
                takeprofit = purps.iloc[i,3]
                stoploss = purps.iloc[i,4]
                CM = purps.iloc[i,5]
                k1 = purps.iloc[i,6]
                k2 = purps.iloc[i,7]
                k3 = purps.iloc[i,8]
                extraj = purps.iloc[i,9]
                RATE, RATE2, spy = livepearls(Xpair,position,entry,takeprofit,stoploss,CM,40,1,k1,k2,k3,extraj)
                purps.iloc[i,6] = RATE
                purps.iloc[i,7] = RATE2
                purps.iloc[i,8] = spy
            i += 1
        watchers = purps['spy']
        print(purps)
        if ite == 280:
            break
        ite += 1
        time.sleep(1)
    
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)
    playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/done.mp3')

In [7]:
print("Initializing...")

pair = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']
levslist = levelgrab(pair)

t2 = 0
while True:
    Drates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("Analyzing Markets...")
            brutus(pair,levslist)
    t2 = t
    time.sleep(1)

Initializing...
Analyzing Markets...
(0.90938, 0.90916, 0.90894, 0.90836, 0.90802, 0.90791, 0.90742, 0.90657, 0.90642, 0.90609, 0.90508, 0.90464, 0.90435, 0.90409, 0.9037, 0.90322, 0.90276, 0.90253, 0.90187, 0.90168, 0.90123, 0.9012, 0.90093, 0.90076, 0.90073, 0.90061, 0.89995, 0.89988, 0.89981, 0.8994, 0.89922, 0.89905, 0.89861, 0.89854, 0.89823, 0.89803, 0.89777, 0.89749, 0.89725, 0.89702, 0.89655, 0.89595, 0.89579, 0.89478, 0.89284, 0.89114)
(0.68218, 0.68206, 0.68131, 0.68107, 0.68037, 0.67993, 0.67934, 0.67892, 0.67854, 0.6782, 0.67773, 0.67751, 0.67743, 0.67724, 0.67713, 0.67693, 0.67649, 0.67642, 0.67605, 0.67567, 0.6753, 0.67519, 0.67499, 0.67463, 0.67386, 0.67364, 0.67269, 0.67265, 0.67252, 0.67209, 0.67183, 0.67039, 0.67005, 0.66994, 0.66915, 0.66743, 0.66588)
(75.242, 75.096, 75.005, 74.794, 74.627, 74.522, 74.498, 74.427, 74.385, 74.344, 74.33, 74.279, 74.271, 74.26, 74.225, 74.196, 74.194, 74.162, 74.091, 74.065, 74.051, 74.017, 73.994, 73.964, 73.954, 73.944, 73.921, 73.8

(72.665, 72.597, 72.438, 72.303, 71.24, 71.108, 71.098, 71.008, 70.952, 70.377, 70.359, 70.251, 70.029, 69.917, 69.802, 69.745, 69.691, 69.604, 69.557, 69.463, 69.392, 69.282, 69.142, 69.12, 69.084, 68.956, 68.05, 68.04, 67.973, 67.602, 67.402, 67.293, 66.123, 65.48, 65.331, 65.255, 65.21, 64.692, 64.261, 64.2, 64.127, 63.919)
(0.66381, 0.66264, 0.66243, 0.66236, 0.66136, 0.66125, 0.65989, 0.65917, 0.64505, 0.64406, 0.6435, 0.6431, 0.64262, 0.64245, 0.6422, 0.64184, 0.64125, 0.64039, 0.63992, 0.63974, 0.63943, 0.63892, 0.63684, 0.63631, 0.63491, 0.63361, 0.63289, 0.63268, 0.63253, 0.63202, 0.63177, 0.63136, 0.63082, 0.63009, 0.62919, 0.62831, 0.62775)
(1.33205, 1.33136, 1.3311, 1.33048, 1.33006, 1.32992, 1.32959, 1.32951, 1.32938, 1.32882, 1.32878, 1.32806, 1.32775, 1.3275000000000001, 1.32714, 1.32699, 1.32682, 1.32617, 1.32596, 1.32537, 1.32498, 1.3243, 1.3241, 1.32349, 1.32344, 1.32281, 1.32207, 1.32084, 1.31998, 1.31954, 1.31942, 1.31914, 1.31786, 1.31772, 1.31722, 1.31392, 1.30687

0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.00017   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.000963   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.27

0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.000949   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533

0.000675   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.000955   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.001003   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  

0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.000302   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.000848   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.2

0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   10

0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.000263   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533

0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   108.367  0.005667     0      0   

   spy      extraj  
0  YES  108.325667  
0.0   seconds
        x position       entry  takeprofit  stoploss        CM  RATE  RATE2  \
0  USDJPY    SHORT  108.331333  108.272533   10

KeyboardInterrupt: 